# Partie 2 - Implementation d'un Arbre de Decision "From Scratch"

Ce notebook presente l'implementation complete d'un arbre de decision binaire sans utiliser de bibliotheques specialisees.

**Contenu:**
- Chargement des donnees depuis GitHub
- Structure de donnees pour l'arbre
- Algorithme de recherche du meilleur split
- Construction recursive de l'arbre
- Fonction de prediction

**Auteur**: Projet Data Mining - Arbres de Decision

## 1. Importation des bibliotheques et chargement des donnees

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Chargement des donnees depuis GitHub
base_url = 'https://raw.githubusercontent.com/NassimZahri/Data_Mining/main/data/'
df = pd.read_csv(base_url + 'credit_simple.csv')

print("Dimensions du dataset:", df.shape)
print("\nApercu des donnees:")
df.head(10)

In [ ]:
# Exploration des donnees
print("Types des colonnes:")
print(df.dtypes)
print("\nValeurs uniques par colonne:")
for col in df.columns:
    print(f"  {col}: {df[col].unique()}")

In [ ]:
# Preparation des donnees
# Separation des features et de la cible
X = df.drop('defaut', axis=1)
y = df['defaut']

# Encodage one-hot des variables categorielles
X_encoded = pd.get_dummies(X)

# Encodage de la variable cible
y_encoded = y.map({'oui': 1, 'non': 0})

print("Features apres encodage:")
print(X_encoded.columns.tolist())
print("\nDimensions:", X_encoded.shape)

## 2. Structure de donnees: Classe Node

Chaque noeud de l'arbre contient:
- Pour un noeud interne: l'attribut de split, le seuil, et les sous-arbres gauche/droit
- Pour une feuille: la valeur de prediction (classe majoritaire)

In [ ]:
class Node:
    """
    Classe representant un noeud dans l'arbre de decision.
    
    Attributs:
    ----------
    feature : str ou None
        Nom de l'attribut utilise pour le split (None si feuille)
    threshold : float ou None
        Valeur seuil pour le split (None si feuille)
    left : Node ou None
        Sous-arbre gauche (instances <= threshold)
    right : Node ou None
        Sous-arbre droit (instances > threshold)
    value : int ou None
        Classe predite (seulement pour les feuilles)
    samples : int
        Nombre d'exemples dans ce noeud
    impurity : float
        Impurete du noeud
    """
    
    def __init__(self, feature=None, threshold=None, left=None, right=None, 
                 value=None, samples=0, impurity=0.0):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.samples = samples
        self.impurity = impurity
    
    def is_leaf(self):
        """Retourne True si le noeud est une feuille."""
        return self.value is not None
    
    def __repr__(self):
        if self.is_leaf():
            return f"Leaf(value={self.value}, samples={self.samples})"
        return f"Node(feature={self.feature}, threshold={self.threshold}, samples={self.samples})"

## 3. Fonction de calcul de l'impurete (Gini)

In [ ]:
def gini(y):
    """
    Calcule l'indice de Gini pour un vecteur de labels.
    
    Parametres:
    -----------
    y : array-like
        Vecteur de labels de classe
    
    Retourne:
    ---------
    float
        Indice de Gini
    """
    if len(y) == 0:
        return 0
    
    counts = Counter(y)
    total = len(y)
    probs = [count / total for count in counts.values()]
    return 1 - sum(p**2 for p in probs)


# Test de la fonction
print("Test de la fonction gini():")
print(f"  Gini([1,1,1,1,0,0,0,0]) = {gini([1,1,1,1,0,0,0,0]):.4f} (attendu: 0.5)")
print(f"  Gini([1,1,1,1,1,1,1,1]) = {gini([1,1,1,1,1,1,1,1]):.4f} (attendu: 0.0)")
print(f"  Gini([1,1,1,1,1,1,0,0]) = {gini([1,1,1,1,1,1,0,0]):.4f}")

## 4. Algorithme de recherche du meilleur split

In [ ]:
def best_split(X, y):
    """
    Trouve le meilleur split pour un noeud donne.
    
    Pour chaque attribut, teste tous les seuils possibles et
    retourne celui qui maximise le gain d'information.
    
    Parametres:
    -----------
    X : DataFrame
        Matrice des features
    y : Series
        Vecteur des labels
    
    Retourne:
    ---------
    tuple (str, float, float)
        (nom_feature, seuil, gain)
    """
    best_gain = 0
    best_feature = None
    best_threshold = None
    
    # Impurete du noeud parent
    parent_impurity = gini(y)
    n_samples = len(y)
    
    # Parcourir tous les attributs
    for feature in X.columns:
        # Obtenir les valeurs uniques triees pour les seuils candidats
        thresholds = sorted(X[feature].unique())
        
        # Tester chaque seuil
        for threshold in thresholds:
            # Diviser les donnees
            left_mask = X[feature] <= threshold
            right_mask = X[feature] > threshold
            
            left_y = y[left_mask]
            right_y = y[right_mask]
            
            # Ignorer les splits qui ne divisent pas les donnees
            if len(left_y) == 0 or len(right_y) == 0:
                continue
            
            # Calculer l'impurete ponderee des enfants
            w_left = len(left_y) / n_samples
            w_right = len(right_y) / n_samples
            
            child_impurity = w_left * gini(left_y) + w_right * gini(right_y)
            
            # Calculer le gain
            gain = parent_impurity - child_impurity
            
            # Mettre a jour le meilleur split
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold
    
    return best_feature, best_threshold, best_gain


# Test de la fonction
print("Test de best_split():")
feature, threshold, gain = best_split(X_encoded, y_encoded)
print(f"  Meilleur split: {feature} <= {threshold} (gain={gain:.4f})")

## 5. Construction recursive de l'arbre

In [ ]:
def build_tree(X, y, depth=0, max_depth=3, min_samples_leaf=1):
    """
    Construit un arbre de decision de maniere recursive.
    
    Parametres:
    -----------
    X : DataFrame
        Matrice des features
    y : Series
        Vecteur des labels
    depth : int
        Profondeur actuelle dans l'arbre
    max_depth : int
        Profondeur maximale autorisee
    min_samples_leaf : int
        Nombre minimum d'echantillons dans une feuille
    
    Retourne:
    ---------
    Node
        Racine de l'arbre (ou sous-arbre)
    """
    n_samples = len(y)
    n_classes = len(set(y))
    impurity = gini(y)
    
    # Conditions d'arret:
    # 1. Noeud pur (une seule classe)
    # 2. Profondeur maximale atteinte
    # 3. Pas assez d'echantillons
    if n_classes == 1 or depth >= max_depth or n_samples < min_samples_leaf * 2:
        # Creer une feuille avec la classe majoritaire
        majority_class = y.mode()[0]
        return Node(value=majority_class, samples=n_samples, impurity=impurity)
    
    # Trouver le meilleur split
    feature, threshold, gain = best_split(X, y)
    
    # Si aucun split ameliore l'impurete, creer une feuille
    if feature is None or gain <= 0:
        majority_class = y.mode()[0]
        return Node(value=majority_class, samples=n_samples, impurity=impurity)
    
    # Diviser les donnees
    left_mask = X[feature] <= threshold
    right_mask = X[feature] > threshold
    
    # Construire recursivement les sous-arbres
    left_subtree = build_tree(
        X[left_mask], y[left_mask], 
        depth + 1, max_depth, min_samples_leaf
    )
    right_subtree = build_tree(
        X[right_mask], y[right_mask], 
        depth + 1, max_depth, min_samples_leaf
    )
    
    # Creer le noeud interne
    return Node(
        feature=feature, 
        threshold=threshold,
        left=left_subtree, 
        right=right_subtree,
        samples=n_samples,
        impurity=impurity
    )

In [ ]:
# Construction de l'arbre
print("Construction de l'arbre de decision...")
tree = build_tree(X_encoded, y_encoded, max_depth=3)
print("Arbre construit avec succes!")
print(f"\nRacine: {tree}")

## 6. Fonction de prediction

In [ ]:
def predict_one(x, node):
    """
    Predit la classe pour une seule instance.
    
    Parametres:
    -----------
    x : Series
        Une instance (ligne du DataFrame)
    node : Node
        Noeud courant de l'arbre
    
    Retourne:
    ---------
    int
        Classe predite (0 ou 1)
    """
    # Si c'est une feuille, retourner la prediction
    if node.is_leaf():
        return node.value
    
    # Sinon, descendre dans le sous-arbre correspondant
    if x[node.feature] <= node.threshold:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)


def predict(X, tree):
    """
    Predit les classes pour un ensemble d'instances.
    
    Parametres:
    -----------
    X : DataFrame
        Matrice des features
    tree : Node
        Racine de l'arbre
    
    Retourne:
    ---------
    list
        Liste des classes predites
    """
    predictions = []
    for idx in X.index:
        pred = predict_one(X.loc[idx], tree)
        predictions.append(pred)
    return predictions

In [ ]:
# Test des predictions
print("Predictions sur l'ensemble d'entrainement:")
y_pred = predict(X_encoded, tree)

# Calcul de l'accuracy
correct = sum(1 for p, t in zip(y_pred, y_encoded) if p == t)
accuracy = correct / len(y_encoded)

print(f"Accuracy: {accuracy:.2%}")
print(f"\nPredictions: {y_pred}")
print(f"Vraies valeurs: {list(y_encoded)}")

## 7. Visualisation de l'arbre

In [ ]:
def print_tree(node, depth=0, prefix="Root"):
    """
    Affiche l'arbre de decision de maniere textuelle.
    
    Parametres:
    -----------
    node : Node
        Noeud courant
    depth : int
        Profondeur actuelle (pour l'indentation)
    prefix : str
        Prefixe pour l'affichage
    """
    indent = "    " * depth
    
    if node.is_leaf():
        class_label = "oui" if node.value == 1 else "non"
        print(f"{indent}{prefix} -> [Defaut: {class_label}] (n={node.samples})")
    else:
        print(f"{indent}{prefix}: {node.feature} <= {node.threshold}? (n={node.samples}, gini={node.impurity:.3f})")
        print_tree(node.left, depth + 1, "Oui")
        print_tree(node.right, depth + 1, "Non")


print("Structure de l'arbre de decision:")
print("=" * 60)
print_tree(tree)

## 8. Matrice de confusion

In [ ]:
def confusion_matrix_manual(y_true, y_pred):
    """
    Calcule la matrice de confusion.
    
    Retourne:
    ---------
    dict
        Dictionnaire avec TP, TN, FP, FN
    """
    tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
    tn = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 0)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)
    fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)
    
    return {'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn}


# Calcul et affichage de la matrice de confusion
cm = confusion_matrix_manual(list(y_encoded), y_pred)

print("Matrice de confusion:")
print("=" * 40)
print(f"                  Prediction")
print(f"                  Non     Oui")
print(f"Reel  Non         {cm['TN']:3d}     {cm['FP']:3d}")
print(f"      Oui         {cm['FN']:3d}     {cm['TP']:3d}")
print()
print(f"Precision: {accuracy:.2%}")
if cm['TP'] + cm['FP'] > 0:
    precision = cm['TP'] / (cm['TP'] + cm['FP'])
    print(f"Precision (positive): {precision:.2%}")
if cm['TP'] + cm['FN'] > 0:
    recall = cm['TP'] / (cm['TP'] + cm['FN'])
    print(f"Rappel: {recall:.2%}")

In [ ]:
print("\nFin du notebook - Arbre de decision from scratch")
print("L'arbre est pret a etre compare avec sklearn dans le notebook suivant.")